In [2]:
import pandas as pd
from pandas.io.json._normalize import nested_to_record  
import  math
import os
import bson
from dateutil import parser
import datetime


from pymongo import MongoClient

def mongoCollection(connstr, db, collection):
    client = MongoClient(connstr)
    db = client[db]
    return db[collection]

In [3]:
collection = mongoCollection(os.environ.get('MONGODB_CONN_STR'), 'smsinfo', 'transactions')

In [5]:
year = int(os.getenv("YEAR", datetime.datetime.now().year))
month = int(os.getenv("MONTH", 3)) #datetime.datetime.now().month))

print("year {year} month {month}".format_map({'year': str(year), 'month': str(month)}))
next_month = 1 if month == 12 else month + 1
next_year = year + 1 if month == 12 else year

monthly = list(collection.find({'status.analysis_done': True,
            'transaction.datetime' : {
                '$gte': datetime.datetime(year,month,1), '$lt': datetime.datetime(next_year,next_month,1)}
            }, {'transaction': 1, 'event' : 1}))

monthly = nested_to_record(monthly, sep='.')

dataframe = pd.DataFrame(monthly)

year 2020 month 3


In [6]:
dataframe

,_id,transaction.expense_amount,transaction.payment_mode,transaction.merchant,transaction.datetime,transaction.category,transaction.sub_category,transaction.comment,transaction.direction,transaction.service_provider,transaction.available_balance,transaction.outstanding_amount,event.name
0,5e5e58e95501e769a54470dc,110.0,PAYTM,Eat Fit,2020-03-01 16:33:01.000,food,delivery,NaN,DEBIT,NaN,NaN,NaN,NaN
1,5e5e58e95501e769a54470e0,329.0,CREDIT Card xx3690,JIO Infocom,2020-03-01 19:07:39.000,utility,mobile,NaN,DEBIT,Jio,273782.78,26217.22,NaN
2,5e5e58ea5501e769a54470ec,500.0,CREDIT Card xx3690,Akshayakalpa,2020-03-02 22:32:37.000,None,None,NaN,TRANSFER,NaN,273282.00,26718.00,NaN
3,5e5e58eb5501e769a54470ee,139.0,PAYTM,Eat Fit,2020-03-03 00:16:07.000,food,delivery,NaN,DEBIT,NaN,NaN,NaN,NaN
4,5e6344530b2e08ee3d72852e,270.0,Akshayakalpa Wallet,Akshayakalpa,2020-03-07 14:10:11.095,grocery,diary,NaN,DEBIT,NaN,260.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,5e85c113e0428e02c2b4e99e,550.0,XXX983,paytmqr2810050501011f9xdbj3suxl@paytm,2020-03-23 00:00:00.000,medicine,medicine,NaN,DEBIT,NaN,NaN,NaN,NaN
81,5e85c380e0428e02c2b4e9a1,22000.0,HDFC,Deepak,2020-03-31 00:00:00.000,Rent,Rent,NaN,DEBIT,Sai krupa heera,NaN,NaN,NaN
82,5e85c580e0428e02c2b4e9a2,400.0,Akshayakalpa Wallet,Akshayakalpa,2020-03-22 16:52:06.466,grocery,diary,NaN,DEBIT,NaN,NaN,NaN,NaN
83,5e85c580e0428e02c2b4e9a3,340.0,Akshayakalpa Wallet,Akshayakalpa,2020-03-27 16:52:06.466,grocery,diary,NaN,DEBIT,NaN,NaN,NaN,NaN
